# lesson-2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[893868, 1124558, 13159297, 1134729, 7145619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[7443040, 115969, 1091997, 1111571, 934112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    
    items = items_weights['item_id']
    probabilities = items_weights['sales_value']
    
    recs = np.random.choice(items, p=probabilities, size=n, replace=False)
    
    return recs.tolist()

In [6]:
%%time

np.random.seed(42)

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['sales_value'] = np.log(items_weights['sales_value'] + 1)
items_weights['sales_value'] = items_weights['sales_value']/sum(items_weights['sales_value'])

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

Wall time: 2.73 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[893868, 1124558, 13159297, 1134729, 7145619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1029125, 13944302, 8203496, 2390413, 893851]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[7443040, 115969, 1091997, 1111571, 934112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[893849, 833577, 12385978, 2507926, 7442915]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[893868, 1124558, 13159297, 1134729, 7145619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1029125, 13944302, 8203496, 2390413, 893851]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[7443040, 115969, 1091997, 1111571, 934112]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[893849, 833577, 12385978, 2507926, 7442915]"


In [8]:
def clear_list(data):
    if type(data) is str:
        result = []
        number = ''
        for i in data[1:-1]:
            if i != ' ' and i != ',' and i != '\n':
                number += i
            elif number != '':
                result.append(number)
                number = ''
        if number != '':
            result.append(number)
        return result
    return data

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(clear_list(bought_list)).astype(float)
    recommended_list = np.array(clear_list(recommended_list)[:k]).astype(float)
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [10]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0003:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.139:tfidf


C:\Users\D9F8~1\AppData\Local\Temp/ipykernel_4644/3957264082.py:7: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.2019:own_purchases
0.0009:weighted_random_recommendation


#### Вывод: лучшее качество показывает алгоритм own_purchases, который предлагает пользователям товары, которые они уже покупали до этого.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### 1). Улучшение бейзлайнов

In [11]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [12]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000[:5]

[6534178, 6533889, 6534166, 6544236, 1404121]

In [13]:
data_train_top_5000 = data_train.loc[data_train['item_id'].isin(top_5000)]
data_train_top_5000.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [14]:
improve_result = data_test.groupby('user_id')['item_id'].unique().reset_index()
improve_result.columns=['user_id', 'actual']
improve_result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### Random recommendation

In [15]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [16]:
%%time

np.random.seed(42)

items = data_train_top_5000.item_id.unique()

improve_result['random_recommendation'] = improve_result['user_id'].apply(lambda x: random_recommendation(items, n=5))

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['random_recommendation'],row['actual'],k=5),axis=1).mean(),4)}")

pr5 - 0.0043
Wall time: 565 ms


#### Popularity-based recommendation

In [17]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [18]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_top_5000, n=5)

improve_result['popular_recommendation'] = improve_result['user_id'].apply(lambda x: popular_recs)

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['popular_recommendation'],row['actual'],k=5),axis=1).mean(),4)}")

pr5 - 0.1552
Wall time: 284 ms


#### Weighted random recommender

In [19]:
%%time

np.random.seed(42)

items_weights = data_train_top_5000.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['sales_value'] = np.log(items_weights['sales_value'] + 1)
items_weights['sales_value'] = items_weights['sales_value']/sum(items_weights['sales_value'])

improve_result['weighted_random_recommendation'] = improve_result['user_id'].apply(lambda x: weighted_random_recommendation(
    items_weights, n=5))

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'],row['actual'],k=5),axis=1).mean(),4)}")

pr5 - 0.0063
Wall time: 634 ms


### 2). Улучшение ItemItem

In [20]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

D:\anakonda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [21]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(5)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#### Item-Item Recommender

In [23]:
%%time

model = ItemItemRecommender(K=18, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


improve_result['itemitem'] = improve_result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean(),4)}")

  0%|          | 0/5001 [00:00<?, ?it/s]

pr5 - 0.1534
Wall time: 1.25 s


#### CosineRecommender

In [24]:
%%time

model = CosineRecommender(K=40, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

improve_result['cosine'] = improve_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], k=5), axis=1).mean(),4)}")

  0%|          | 0/5001 [00:00<?, ?it/s]

pr5 - 0.1405
Wall time: 1.48 s


#### TFIDFRecommender

In [25]:
%%time

model = TFIDFRecommender(K=30, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

improve_result['tfidf'] = improve_result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

print(f"pr5 - {round(improve_result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean(),4)}")

  0%|          | 0/5001 [00:00<?, ?it/s]

pr5 - 0.1506
Wall time: 1.44 s


#### Сравнение метрик

In [26]:
improve_result["own_purchases"] = result["own_purchases"]

In [27]:
improve_result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6463729, 924834, 1118787, 968687, 1057377]","[6534178, 6533889, 1029743, 6534166, 1082185]","[958137, 9858742, 1104414, 1048352, 873654]","[999999, 1082185, 981760, 995242, 840361]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 1127831, 840361, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[913202, 869024, 1109374, 938841, 1011884]","[6534178, 6533889, 1029743, 6534166, 1082185]","[873654, 839346, 6464166, 1048893, 1094767]","[999999, 1082185, 981760, 1098066, 826249]","[1082185, 1098066, 826249, 981760, 999999]","[1082185, 1098066, 826249, 981760, 1044078]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [28]:
print("old : improve : algorithm\n")
for name_col in improve_result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5), axis=1).mean(),4)}"
          f" : {round(improve_result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5), axis=1).mean(),4)}"
          f" : {name_col}")

old : improve : algorithm

1.0 : 1.0 : actual
0.0003 : 0.0043 : random_recommendation
0.1552 : 0.1552 : popular_recommendation
0.0009 : 0.0063 : weighted_random_recommendation
0.1369 : 0.1534 : itemitem
0.1329 : 0.1405 : cosine
0.139 : 0.1506 : tfidf


C:\Users\D9F8~1\AppData\Local\Temp/ipykernel_4644/3957264082.py:7: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.2019 : 0.2019 : own_purchases


#### Вывод: метрики заметно улучшились, но алгоритм own_purchases, который предлагает пользователям товары, которые они уже покупали до этого, остался наилучшим.